# ACL Spotify Genre Finder

#### The purpose of this workbook is to find all of the genres associated with the artists on the ACL playlist, created using the ACL_Playlist_Creator workbook.

##### Import Spotify Data and Modules

In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd

##### Initialize Spotify Developer Tools


In [2]:
client_id = "hidden" # I've hidden my ID in order to publish this code
client_secret = "hidden" # I've hidden my secret in order to publish this code
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

##### Loop through JSON of playlist info to extract all tracks on the playlist


In [3]:
def get_playlist_tracks(playlist_id):
    '''Spotipy method of returning list of playlist song features stored in a dictionary

    Keyword arguments:
    playlist_id -- the playlist URI
    '''
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

urn = 'spotify:playlist:0TCMt3rZMXJfOfvNBDf34E' # Spotify code for the playlist

playlist = get_playlist_tracks(urn)

##### Loop through playlist tracks to extract key elements


In [6]:
df_list = []
genre_df_list = []
artist_id_list = []
artist_genre_list = []

for index, val in enumerate(playlist):
    #Collects song information for each track in playlist
    track = playlist[index]['track']['name']
    track_id = playlist[index]['track']['id']
    artist = playlist[index]['track']['album']['artists'][0]['name']
    artist_id = playlist[index]['track']['album']['artists'][0]['id']
    song_id = 'spotify:track:' + track_id 
    artist_id = 'spotify:artist:' + artist_id
    # Collects up to 50 unique artists for Spotipy artist info API call
    if artist_id not in artist_id_list:
        artist_id_list.append(artist_id)
    if len(artist_id_list) == 50 or index == len(playlist) - 1:
        artist_info_dict = sp.artists(artist_id_list)
        # Collects artist genre information
        for index, val in enumerate(artist_info_dict['artists']):
            genres = artist_info_dict['artists'][index]['genres']
            artist = artist_info_dict['artists'][index]['name']
            artist_genre_list.append([artist, genres])
            artist_id_list = []
    df_list.append([track, artist, artist_id, track_id])

##### Convert playlist features into a dataframe


In [7]:
df = pd.DataFrame(df_list, columns=['Track', 'Artist', 'Artist ID', 'Track ID'])

##### Convert artist genre info into a dataframe


In [8]:
genre_df = pd.DataFrame(artist_genre_list, columns=['Artist', 'Genre'])

##### Merge playlist and genre dataframes


In [9]:
df = pd.merge(df, genre_df)

##### Explode column of genre lists into a new dataframe


In [10]:
genre_by_artist_df = df.drop_duplicates(subset=['Artist'], keep='first').explode('Genre')[['Artist', 'Genre']].dropna().reset_index(drop=True)


##### Save genre dataframe as csv in order to merge with origianal artist data

In [13]:
genre_by_artist_df.to_csv('ACL_Genre_Text.csv')